# Flight Data Files to CSV
Here we create the CSV file of per-flight data to import into our WordPress website.  By creating flights (technically, WordPress posts of `post_type` = `flight`) from CSV, I've been able to iterate dozens of times, trying different datatypes, etc.  New fields can be easily added, again, without having to enter data into forms by hand.  **The saga of WordPress and datatypes is far too long to describe here, but is very imporant you understand them...**

Per-flight metadata (flight date, altitude, pilots, etc.) came mostly from Jackie's spreadsheet, while data generated and collected during each flight was gathered from several different ad-hoc locations, including Morgan's SVN repo and https://perlanproject.cloud/data, and organized into one directory per flight, with standardized filenames.  For example, the KML file for Flight 0065 is named `/data/Flights/0069/Flt0069.kml`.  

NOTES:

- The web hostname must be prepended to make a full-path URI, e.g., `http://localhost/data/Flights/0069/Flt0069.kml`.  For development, hostname is `http://localhost`.  When the site is deployed, the hostname is changed to reflect this, either manually (if you're a stud) or via WordPress plugin.  Using our example, `https://perlanproject.org/data/Flights/0069/Flt0069.kml`

- The `data` directory lives at the top level (root) of the website, or more precisely, is *accessible* from the root; it is actually symlinked so it can be backed up in a Git repo using Large File Support (LFS), as there are some big honkin' files.

- We actually read *and* write `flights.csv` and then *write* `flights-toolset.csv`, the latter being the file we import into WordPress to create Flights (`post_type == 'flight'`) as implemented by the Toolset plugin.  Toolset is a paid (tho inexpensive) WordPress plugin that implements the custom data types, custom fields, custom post types, etc. we need.  WordPress itself is **very** bare-bones, as it was designed as a blogging platform, not a Content Management System (CMS).  At least that the excuse they give.  IMHO, WordPress would be a **lot** more stable, more powerful, more usable, and all-around have much better karma, had the WordPress developers decided to add all these missing pieces directly into the platform at some point, rather than let the hodge-podge we have today happen.  But nooooooooo....


In [108]:
import os
import pandas as pd
import path_utils as pu

In [109]:
root = "/Users/jdm/workbench/Perlan" # YMMV
os.chdir(root)

In [110]:
!pwd

/Users/jdm/workbench/Perlan


In [111]:
!ls

Controlled.svn                    data_website.202105026_1010
Perlan Encore Fellowship          data_website.202105026_1010.zip
PerlanProject-2020-07-07T19-16-38 data_website.drupaled.broken
Science.git                       data_website.duplicator.archives
TRASH_LATER                       perlanproject.org
Toolset                           pods
assets                            pods.old
clippings                         tmp
data                              windField
data website plan.ooutline        wp-config.php.save
data_website                      wp_options__wpcf_fields


# Read CSV

We expect `flights.csv` to have the per-flight metadata already.  Originally it was gleaned from `flights.xlsx` but now has been further modified.

After reading `flights.csv` we walk the directory `data/Flights` to find data files and add those names to the CSV.

In [112]:
svn_root  = 'Controlled.svn/Systems/Data Network Logs/'
data_root = 'data/'
flights_root = data_root + 'Flights'
ballons_root = data_root + 'Soundings'

In [173]:
csv = pd.read_csv(f"{data_root}/flights.csv")

In [174]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'flight_date', 'city',
       'flight_number', 'takeoff_airport', 'takeoff_time_local',
       'landing_airport', 'landing_time_local', 'duration',
       'release_altitude_feet', 'maximum_altitude_feet',
       'maximum_gps_altitude_feet', 'pilot_front', 'pilot_rear', 'data_ac',
       'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

## Initial Cleanup


In [115]:
csv['post_name'] = csv.flight_number.apply(lambda s: f"Flight {s:04}")

In [116]:
# Zero-out the columns we populate by walking the directory tree to find files
csv['data_ac'] = None
csv['data_uv'] = None
csv['data_kml'] = None
csv['data_adp'] = None
csv['data_imu'] = None

In [117]:
# Delete 'Unnamed' columns - this happens if CSV is saved via df.to_csv(index=True)
# This shouldn't happen, of course, and it's harmless, but annoying.
for col in csv.columns:
    match = 'Unnamed'
    if col[:len(match)] == match:
        print(f"Deleting junk column {col}")
        del csv[col]

In [118]:
# delete any null rows
csv.dropna(how='all', inplace=True)

In [119]:
# ensure correct types - int columns can become float if any missing data
csv['flight_number'] = csv.flight_number.astype(int, copy=False)

## Sanity Check: Input
Look these over to make sure everything looks OK.

In [120]:
csv.head()

,post_name,post_post,post_excerpt,flight_number,takeoff_airport,takeoff_time_local,landing_airport,landing_time_local,duration,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_uv,data_kml,data_adp,data_imu
0,Flight 0001,First Flight,First Flight,1,"KRDM (Redmond, OR, USA)",818.0,"KRDM (Redmond, OR, USA)",851.0,0.6,8100.0,8100.0,NaN,Jim Payne,Morgan Sandercock,None,None,None,None,None
1,Flight 0002,flight testing,flight testing,2,"KMEV (Minden, NV, USA)",1307.0,"KMEV (Minden, NV, USA)",1350.0,0.7,10800.0,10800.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
2,Flight 0003,NaN,NaN,3,"KMEV (Minden, NV, USA)",1420.0,"KMEV (Minden, NV, USA)",1502.0,0.7,10700.0,10700.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
3,Flight 0004,NaN,NaN,4,"KMEV (Minden, NV, USA)",1423.0,"KMEV (Minden, NV, USA)",1445.0,0.4,7600.0,7600.0,NaN,Jim Payne,NaN,None,None,None,None,None
4,Flight 0005,NaN,NaN,5,"KMEV (Minden, NV, USA)",1525.0,"KMEV (Minden, NV, USA)",1555.0,0.5,8700.0,8700.0,NaN,Jim Payne,NaN,None,None,None,None,None


In [121]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'flight_number',
       'takeoff_airport', 'takeoff_time_local', 'landing_airport',
       'landing_time_local', 'duration', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

In [122]:
rec = csv[csv['flight_number'] == 65]
print(rec)

      post_name            post_post         post_excerpt  flight_number  \
64  Flight 0065  Last flight of 2019  Last flight of 2019             65   

                  takeoff_airport  takeoff_time_local  \
64  SAWC (El Calafate, Argentina)              1120.0   

                  landing_airport  landing_time_local  duration  \
64  SAWC (El Calafate, Argentina)              1650.0       5.5   

    release_altitude_feet  maximum_altitude_feet  maximum_gps_altitude_feet  \
64                51000.0                65000.0                        NaN   

   pilot_front         pilot_rear data_ac data_uv data_kml data_adp data_imu  
64   Jim Payne  Morgan Sandercock    None    None     None     None     None  


In [123]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'flight_number',
       'takeoff_airport', 'takeoff_time_local', 'landing_airport',
       'landing_time_local', 'duration', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

# Do the Work

In [124]:
flts = pu.get_subdirs(flights_root)
#flts

In [125]:
# Walk the Flights dir, looking for data files, and put them in the CSV
def stuff_file_paths(df=None, flight_dirs=None, host="http://localhost", verbose=1):
    for flt in flight_dirs:
        files_full = pu.get_files(flt)
        files = [f[f.rfind('/')+1:] for f in files_full]
        if files == []:
            continue
        
        nr = int(flt[-4:])
        if verbose:
            print(f"#{nr}\t{flt}\t{files}")

        for file in files_full:
            file = f'{host}/{file}'
            base = file[:file.rfind('.')]
            suffix = file[file.rfind('.')+1:]
            idx = nr - 1
            #if verbose:
            #    print(f"base = {base}   suffix = {suffix}")
            if suffix == 'kml':
                df.loc[idx, 'data_kml'] = file
                if verbose:
                    print(f"KML = {file}")
            #print(f"base[-2:]={base[-2:]}")                    
            for kind in ['AC', 'ADP', 'IMU', 'UV']:
                if base[-len(kind):] == kind:
                    df.loc[idx, f'data_{kind.lower()}'] = file
                    if verbose:
                        print(f"{kind} = {file}")                
    return df

stuff_file_paths(df=csv, flight_dirs=flts)

#61	data/Flights/0061	['Flt0061IMU.zip', 'Flt0061AC.xlsb', 'Flt0061.kml', 'Flt0061ADP.csv', 'Flt0061UV.xlsx']
IMU = http://localhost/data/Flights/0061/Flt0061IMU.zip
AC = http://localhost/data/Flights/0061/Flt0061AC.xlsb
KML = http://localhost/data/Flights/0061/Flt0061.kml
ADP = http://localhost/data/Flights/0061/Flt0061ADP.csv
UV = http://localhost/data/Flights/0061/Flt0061UV.xlsx
#59	data/Flights/0059	['Flt0059AC.xlsx', 'Flt0059IMU.zip', 'Flt0059ADP.csv', 'Flt0059.kml', 'Flt0059UV.xlsx']
AC = http://localhost/data/Flights/0059/Flt0059AC.xlsx
IMU = http://localhost/data/Flights/0059/Flt0059IMU.zip
ADP = http://localhost/data/Flights/0059/Flt0059ADP.csv
KML = http://localhost/data/Flights/0059/Flt0059.kml
UV = http://localhost/data/Flights/0059/Flt0059UV.xlsx
#50	data/Flights/0050	['Flt0050UV.xlsx', 'Flt0050.kml', 'Flt0050AC.xlsb']
UV = http://localhost/data/Flights/0050/Flt0050UV.xlsx
KML = http://localhost/data/Flights/0050/Flt0050.kml
AC = http://localhost/data/Flights/0050/Flt0050A

,post_name,post_post,post_excerpt,flight_number,takeoff_airport,takeoff_time_local,landing_airport,landing_time_local,duration,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_uv,data_kml,data_adp,data_imu
0,Flight 0001,First Flight,First Flight,1,"KRDM (Redmond, OR, USA)",818.0,"KRDM (Redmond, OR, USA)",851.0,0.6,8100.0,8100.0,NaN,Jim Payne,Morgan Sandercock,None,None,None,None,None
1,Flight 0002,flight testing,flight testing,2,"KMEV (Minden, NV, USA)",1307.0,"KMEV (Minden, NV, USA)",1350.0,0.7,10800.0,10800.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
2,Flight 0003,NaN,NaN,3,"KMEV (Minden, NV, USA)",1420.0,"KMEV (Minden, NV, USA)",1502.0,0.7,10700.0,10700.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
3,Flight 0004,NaN,NaN,4,"KMEV (Minden, NV, USA)",1423.0,"KMEV (Minden, NV, USA)",1445.0,0.4,7600.0,7600.0,NaN,Jim Payne,NaN,None,None,None,None,None
4,Flight 0005,NaN,NaN,5,"KMEV (Minden, NV, USA)",1525.0,"KMEV (Minden, NV, USA)",1555.0,0.5,8700.0,8700.0,NaN,Jim Payne,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Flight 0061,NaN,NaN,61,"SAWC (El Calafate, Argentina)",1037.0,"SAWC (El Calafate, Argentina)",1430.0,3.9,46600.0,56300.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0061/Flt0061AC.xlsb,http://localhost/data/Flights/0061/Flt0061UV.xlsx,http://localhost/data/Flights/0061/Flt0061.kml,http://localhost/data/Flights/0061/Flt0061ADP.csv,http://localhost/data/Flights/0061/Flt0061IMU.zip
61,Flight 0062,NaN,NaN,62,"SAWC (El Calafate, Argentina)",1333.0,"SAWC (El Calafate, Argentina)",1636.0,3.1,42000.0,42300.0,NaN,Jim Payne,Morgan Sandercock,http://localhost/data/Flights/0062/Flt0062AC.xlsb,http://localhost/data/Flights/0062/Flt0062UV.xlsx,http://localhost/data/Flights/0062/Flt0062.kml,http://localhost/data/Flights/0062/Flt0062ADP.csv,http://localhost/data/Flights/0062/Flt0062IMU.zip
62,Flight 0063,tow height record,tow height record,63,"SAWC (El Calafate, Argentina)",1235.0,"SAWC (El Calafate, Argentina)",1811.0,5.6,47100.0,50600.0,NaN,Jim Payne,Tim Gardner,http://localhost/data/Flights/0063/Flt0063AC.xlsb,http://localhost/data/Flights/0063/Flt0063UV.xlsb,http://localhost/data/Flights/0063/Flt0063.kml,http://localhost/data/Flights/0063/Flt0063ADP.zip,http://localhost/data/Flights/0063/Flt0063IMU.zip
63,Flight 0064,NaN,NaN,64,"SAWC (El Calafate, Argentina)",913.0,"SAWC (El Calafate, Argentina)",1230.0,3.3,45100.0,49200.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0064/Flt0064AC.xlsb,http://localhost/data/Flights/0064/Flt0064UV.xlsb,http://localhost/data/Flights/0064/Flt0064.kml,http://localhost/data/Flights/0064/Flt0064ADP.csv,http://localhost/data/Flights/0064/Flt0064IMU.zip


# Sanity Check: Output

In [126]:
csv

,post_name,post_post,post_excerpt,flight_number,takeoff_airport,takeoff_time_local,landing_airport,landing_time_local,duration,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_uv,data_kml,data_adp,data_imu
0,Flight 0001,First Flight,First Flight,1,"KRDM (Redmond, OR, USA)",818.0,"KRDM (Redmond, OR, USA)",851.0,0.6,8100.0,8100.0,NaN,Jim Payne,Morgan Sandercock,None,None,None,None,None
1,Flight 0002,flight testing,flight testing,2,"KMEV (Minden, NV, USA)",1307.0,"KMEV (Minden, NV, USA)",1350.0,0.7,10800.0,10800.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
2,Flight 0003,NaN,NaN,3,"KMEV (Minden, NV, USA)",1420.0,"KMEV (Minden, NV, USA)",1502.0,0.7,10700.0,10700.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
3,Flight 0004,NaN,NaN,4,"KMEV (Minden, NV, USA)",1423.0,"KMEV (Minden, NV, USA)",1445.0,0.4,7600.0,7600.0,NaN,Jim Payne,NaN,None,None,None,None,None
4,Flight 0005,NaN,NaN,5,"KMEV (Minden, NV, USA)",1525.0,"KMEV (Minden, NV, USA)",1555.0,0.5,8700.0,8700.0,NaN,Jim Payne,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Flight 0061,NaN,NaN,61,"SAWC (El Calafate, Argentina)",1037.0,"SAWC (El Calafate, Argentina)",1430.0,3.9,46600.0,56300.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0061/Flt0061AC.xlsb,http://localhost/data/Flights/0061/Flt0061UV.xlsx,http://localhost/data/Flights/0061/Flt0061.kml,http://localhost/data/Flights/0061/Flt0061ADP.csv,http://localhost/data/Flights/0061/Flt0061IMU.zip
61,Flight 0062,NaN,NaN,62,"SAWC (El Calafate, Argentina)",1333.0,"SAWC (El Calafate, Argentina)",1636.0,3.1,42000.0,42300.0,NaN,Jim Payne,Morgan Sandercock,http://localhost/data/Flights/0062/Flt0062AC.xlsb,http://localhost/data/Flights/0062/Flt0062UV.xlsx,http://localhost/data/Flights/0062/Flt0062.kml,http://localhost/data/Flights/0062/Flt0062ADP.csv,http://localhost/data/Flights/0062/Flt0062IMU.zip
62,Flight 0063,tow height record,tow height record,63,"SAWC (El Calafate, Argentina)",1235.0,"SAWC (El Calafate, Argentina)",1811.0,5.6,47100.0,50600.0,NaN,Jim Payne,Tim Gardner,http://localhost/data/Flights/0063/Flt0063AC.xlsb,http://localhost/data/Flights/0063/Flt0063UV.xlsb,http://localhost/data/Flights/0063/Flt0063.kml,http://localhost/data/Flights/0063/Flt0063ADP.zip,http://localhost/data/Flights/0063/Flt0063IMU.zip
63,Flight 0064,NaN,NaN,64,"SAWC (El Calafate, Argentina)",913.0,"SAWC (El Calafate, Argentina)",1230.0,3.3,45100.0,49200.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0064/Flt0064AC.xlsb,http://localhost/data/Flights/0064/Flt0064UV.xlsb,http://localhost/data/Flights/0064/Flt0064.kml,http://localhost/data/Flights/0064/Flt0064ADP.csv,http://localhost/data/Flights/0064/Flt0064IMU.zip


In [127]:
# rename any columns, as needed.  I often forget the params for df.rename, so
# keeping an example here is handy as a crutch for my age-addled brain.
if False:
    csv.rename(mapper={'pic':'pilot_front','sic':'pilot_rear'}, axis=1, inplace=True)
    csv.columns

In [128]:
for kind in ['AC', 'ADP', 'KML', 'IMU', 'UV']:
    print(f"Non-null entries for {kind}: {len(csv[csv[f'data_{kind.lower()}'].notnull()])}")

Non-null entries for AC: 43
Non-null entries for ADP: 8
Non-null entries for KML: 29
Non-null entries for IMU: 10
Non-null entries for UV: 20


In [129]:
csv[ csv['data_kml'].notnull()][['data_ac', 'data_adp', 'data_kml', 'data_imu', 'data_uv']]

,data_ac,data_adp,data_kml,data_imu,data_uv
20,http://localhost/data/Flights/0021/Flt0021AC.xlsx,None,http://localhost/data/Flights/0021/Flt0021.kml,None,None
21,http://localhost/data/Flights/0022/Flt0022AC.xlsx,None,http://localhost/data/Flights/0022/Flt0022.kml,None,None
22,http://localhost/data/Flights/0023/Flt0023AC.xlsx,None,http://localhost/data/Flights/0023/Flt0023.kml,None,None
24,http://localhost/data/Flights/0025/Flt0025AC.xlsx,None,http://localhost/data/Flights/0025/Flt0025.kml,None,None
25,http://localhost/data/Flights/0026/Flt0025AC.xlsx,None,http://localhost/data/Flights/0026/Flt0026.kml,None,None
30,http://localhost/data/Flights/0031/Flt0031AC.xlsx,None,http://localhost/data/Flights/0031/Flt0031.kml,None,None
31,http://localhost/data/Flights/0032/Flt0032AC.xlsb,None,http://localhost/data/Flights/0032/Flt0032.kml,None,None
32,http://localhost/data/Flights/0033/Flt0033AC.xlsb,None,http://localhost/data/Flights/0033/Flt0033.kml,None,None
34,http://localhost/data/Flights/0035/Flt0035AC.xlsb,None,http://localhost/data/Flights/0035/Flt0035.kml,None,None
35,http://localhost/data/Flights/0036/Flt0036AC.xlsb,None,http://localhost/data/Flights/0036/Flt0036.kml,None,None


In [130]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'flight_number',
       'takeoff_airport', 'takeoff_time_local', 'landing_airport',
       'landing_time_local', 'duration', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

In [131]:
csv.dtypes

post_name                     object
post_post                     object
post_excerpt                  object
flight_number                  int64
takeoff_airport               object
takeoff_time_local           float64
landing_airport               object
landing_time_local           float64
duration                     float64
release_altitude_feet        float64
maximum_altitude_feet        float64
maximum_gps_altitude_feet    float64
pilot_front                   object
pilot_rear                    object
data_ac                       object
data_uv                       object
data_kml                      object
data_adp                      object
data_imu                      object
dtype: object

In [132]:
csv.flight_number  # index is zero-based, thus (flight_number - 1)

0      1
1      2
2      3
3      4
4      5
      ..
60    61
61    62
62    63
63    64
64    65
Name: flight_number, Length: 65, dtype: int64

In [133]:
csv.pilot_rear.unique()

array(['Morgan Sandercock', 'Miguel Iturmendi', nan, 'Tom Enders',
       'Doug Perrenod', 'Tim Gardner'], dtype=object)

In [134]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'flight_number',
       'takeoff_airport', 'takeoff_time_local', 'landing_airport',
       'landing_time_local', 'duration', 'release_altitude_feet',
       'maximum_altitude_feet', 'maximum_gps_altitude_feet', 'pilot_front',
       'pilot_rear', 'data_ac', 'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

In [135]:
csv.takeoff_airport.unique()

array(['KRDM (Redmond, OR, USA)', 'KMEV (Minden, NV, USA)',
       'SAWC (El Calafate, Argentina)'], dtype=object)

In [136]:
csv.iloc[0]

post_name                                Flight 0001
post_post                               First Flight
post_excerpt                            First Flight
flight_number                                      1
takeoff_airport              KRDM (Redmond, OR, USA)
takeoff_time_local                             818.0
landing_airport              KRDM (Redmond, OR, USA)
landing_time_local                             851.0
duration                                         0.6
release_altitude_feet                         8100.0
maximum_altitude_feet                         8100.0
maximum_gps_altitude_feet                        NaN
pilot_front                                Jim Payne
pilot_rear                         Morgan Sandercock
data_ac                                         None
data_uv                                         None
data_kml                                        None
data_adp                                        None
data_imu                                      

In [137]:
csv

,post_name,post_post,post_excerpt,flight_number,takeoff_airport,takeoff_time_local,landing_airport,landing_time_local,duration,release_altitude_feet,maximum_altitude_feet,maximum_gps_altitude_feet,pilot_front,pilot_rear,data_ac,data_uv,data_kml,data_adp,data_imu
0,Flight 0001,First Flight,First Flight,1,"KRDM (Redmond, OR, USA)",818.0,"KRDM (Redmond, OR, USA)",851.0,0.6,8100.0,8100.0,NaN,Jim Payne,Morgan Sandercock,None,None,None,None,None
1,Flight 0002,flight testing,flight testing,2,"KMEV (Minden, NV, USA)",1307.0,"KMEV (Minden, NV, USA)",1350.0,0.7,10800.0,10800.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
2,Flight 0003,NaN,NaN,3,"KMEV (Minden, NV, USA)",1420.0,"KMEV (Minden, NV, USA)",1502.0,0.7,10700.0,10700.0,NaN,Jim Payne,Miguel Iturmendi,None,None,None,None,None
3,Flight 0004,NaN,NaN,4,"KMEV (Minden, NV, USA)",1423.0,"KMEV (Minden, NV, USA)",1445.0,0.4,7600.0,7600.0,NaN,Jim Payne,NaN,None,None,None,None,None
4,Flight 0005,NaN,NaN,5,"KMEV (Minden, NV, USA)",1525.0,"KMEV (Minden, NV, USA)",1555.0,0.5,8700.0,8700.0,NaN,Jim Payne,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Flight 0061,NaN,NaN,61,"SAWC (El Calafate, Argentina)",1037.0,"SAWC (El Calafate, Argentina)",1430.0,3.9,46600.0,56300.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0061/Flt0061AC.xlsb,http://localhost/data/Flights/0061/Flt0061UV.xlsx,http://localhost/data/Flights/0061/Flt0061.kml,http://localhost/data/Flights/0061/Flt0061ADP.csv,http://localhost/data/Flights/0061/Flt0061IMU.zip
61,Flight 0062,NaN,NaN,62,"SAWC (El Calafate, Argentina)",1333.0,"SAWC (El Calafate, Argentina)",1636.0,3.1,42000.0,42300.0,NaN,Jim Payne,Morgan Sandercock,http://localhost/data/Flights/0062/Flt0062AC.xlsb,http://localhost/data/Flights/0062/Flt0062UV.xlsx,http://localhost/data/Flights/0062/Flt0062.kml,http://localhost/data/Flights/0062/Flt0062ADP.csv,http://localhost/data/Flights/0062/Flt0062IMU.zip
62,Flight 0063,tow height record,tow height record,63,"SAWC (El Calafate, Argentina)",1235.0,"SAWC (El Calafate, Argentina)",1811.0,5.6,47100.0,50600.0,NaN,Jim Payne,Tim Gardner,http://localhost/data/Flights/0063/Flt0063AC.xlsb,http://localhost/data/Flights/0063/Flt0063UV.xlsb,http://localhost/data/Flights/0063/Flt0063.kml,http://localhost/data/Flights/0063/Flt0063ADP.zip,http://localhost/data/Flights/0063/Flt0063IMU.zip
63,Flight 0064,NaN,NaN,64,"SAWC (El Calafate, Argentina)",913.0,"SAWC (El Calafate, Argentina)",1230.0,3.3,45100.0,49200.0,NaN,Jim Payne,Miguel Iturmendi,http://localhost/data/Flights/0064/Flt0064AC.xlsb,http://localhost/data/Flights/0064/Flt0064UV.xlsb,http://localhost/data/Flights/0064/Flt0064.kml,http://localhost/data/Flights/0064/Flt0064ADP.csv,http://localhost/data/Flights/0064/Flt0064IMU.zip


# Write Generic CSV
This CSV is not Toolset-specific (see below)

In [178]:
# Generic CSV

csv.to_csv(f"{data_root}/flights.csv", index=False)
print("generic CSV saved!")

generic CSV saved!


In [179]:
csv.columns

Index(['post_name', 'post_post', 'post_excerpt', 'post_type', 'flight_date',
       'city', 'flight_number', 'takeoff_airport', 'takeoff_time_local',
       'landing_airport', 'landing_time_local', 'duration',
       'release_altitude_feet', 'maximum_altitude_feet',
       'maximum_gps_altitude_feet', 'pilot_front', 'pilot_rear', 'data_ac',
       'data_uv', 'data_kml', 'data_adp', 'data_imu'],
      dtype='object')

In [181]:
csv.iloc[64]

post_name                                                          Flight 0065
post_post                                                  Last flight of 2019
post_excerpt                                               Last flight of 2019
post_type                                                               flight
flight_date                                                         2019-09-17
city                                                               El Calafate
flight_number                                                               65
takeoff_airport                                  SAWC (El Calafate, Argentina)
takeoff_time_local                                                      1120.0
landing_airport                                  SAWC (El Calafate, Argentina)
landing_time_local                                                      1650.0
duration                                                                   5.5
release_altitude_feet                               

# CSV for Toolset, to be imported by "CSV Importer"
For import in the Toolset plugin Types module by the "CSV Importer" plugin.

- CSV Importer requires WordPress post fields prepended with `csv_`, and requires these fields (columns):
    - csv_post_title
    - csv_post_post
    - csv_post_type
    - csv_post_excerpt
    - csv_post_categories
    - csv_post_tags
    - csv_post_date: NOTE: this is just a PHP string, unlike Toolset, which uses POSIX timestamp format
- See https://wordpress.org/plugins/csv-importer/ for more details on CSV Importer
- Toolset requires custom post fields prepended with `wpcf-` (the WPCF plugin was Toolset's first?)
- Date/time fields in POSIX timestamp format (float)

In [182]:
cols = list(csv.columns)
cols

['post_name',
 'post_post',
 'post_excerpt',
 'post_type',
 'flight_date',
 'city',
 'flight_number',
 'takeoff_airport',
 'takeoff_time_local',
 'landing_airport',
 'landing_time_local',
 'duration',
 'release_altitude_feet',
 'maximum_altitude_feet',
 'maximum_gps_altitude_feet',
 'pilot_front',
 'pilot_rear',
 'data_ac',
 'data_uv',
 'data_kml',
 'data_adp',
 'data_imu']

In [183]:
df = pd.DataFrame()

In [187]:
for col in cols:
    if col[:len('post_')] == 'post_':
        df[f"csv_{col}"] = csv[col]     # note the underscore in 'csv_'
    else:
        df[f"wpcf-{col}"] = csv[col]    # note the dash in 'wpcf-'
    

In [188]:
df.columns

Index(['csv_post_name', 'csv_post_post', 'csv_post_excerpt', 'csv_post_type',
       'wpcf-flight_date', 'wpcf-city', 'wpcf-flight_number',
       'wpcf-takeoff_airport', 'wpcf-takeoff_time_local',
       'wpcf-landing_airport', 'wpcf-landing_time_local', 'wpcf-duration',
       'wpcf-release_altitude_feet', 'wpcf-maximum_altitude_feet',
       'wpcf-maximum_gps_altitude_feet', 'wpcf-pilot_front', 'wpcf-pilot_rear',
       'wpcf-data_ac', 'wpcf-data_uv', 'wpcf-data_kml', 'wpcf-data_adp',
       'wpcf-data_imu'],
      dtype='object')

In [191]:
i = df.iloc[64]
i

csv_post_name                                                           Flight 0065
csv_post_post                                                   Last flight of 2019
csv_post_excerpt                                                Last flight of 2019
csv_post_type                                                                flight
wpcf-flight_date                                                         2019-09-17
wpcf-city                                                               El Calafate
wpcf-flight_number                                                               65
wpcf-takeoff_airport                                  SAWC (El Calafate, Argentina)
wpcf-takeoff_time_local                                                      1120.0
wpcf-landing_airport                                  SAWC (El Calafate, Argentina)
wpcf-landing_time_local                                                      1650.0
wpcf-duration                                                               

In [193]:
from datetime import datetime as dt

In [200]:
dt.strptime(i['wpcf-flight_date'] + '-' + str(int(i['wpcf-takeoff_time_local'])), "%Y-%m-%d-%H%M")

datetime.datetime(2019, 9, 17, 11, 20)

In [146]:
# Toolset-specific CSV

df.to_csv(f"{data_root}/flights-toolset.csv", index=False)
print("Toolset CSV saved!")

Toolset CSV saved!
